In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.multioutput import MultiOutputRegressor

# 读取数据
features = pd.read_excel('数据融合--高温合金数据库.xlsx')

# 提取目标变量 UTS、EL、TYS 和 RA
labels = features[['UTS', 'EL', 'TYS', 'RA']].values

# 删除目标变量及其他不需要的特征
features = features.drop(['UTS', 'EL', 'TYS', 'RA', '合金牌号', '材料', '熔炼工艺'], axis=1)

# 对特征进行独热编码
features = pd.get_dummies(features)
feature_list = list(features.columns)
features = np.array(features)

# 归一化处理
scaler = MinMaxScaler()
features = scaler.fit_transform(features)

# 划分训练集、验证集和测试集
test_ratio = 0.25
validation_ratio = 0.2  # 20% 的数据用于验证集合
SEED = 26

# 划分训练集和临时集
train_features, temp_features, train_labels, temp_labels = train_test_split(
    features, labels, test_size=(test_ratio + validation_ratio), random_state=SEED)

# 划分临时集为测试集和验证集
test_features, validation_features, test_labels, validation_labels = train_test_split(
    temp_features, temp_labels, 
    test_size=validation_ratio / (test_ratio + validation_ratio),
    random_state=SEED)

# 创建并拟合多目标SVR模型
base_svr = SVR(C=2000, kernel='rbf', gamma=0.4444443333333)
model_mosvr = MultiOutputRegressor(base_svr)
model_mosvr.fit(train_features, train_labels)

# 预测结果
train_predicted = model_mosvr.predict(train_features)
test_predicted = model_mosvr.predict(test_features)
validation_predicted = model_mosvr.predict(validation_features)

# 分别提取各目标变量的预测结果
train_predicted_UTS, train_predicted_EL, train_predicted_TYS, train_predicted_RA = train_predicted.T
test_predicted_UTS, test_predicted_EL, test_predicted_TYS, test_predicted_RA = test_predicted.T
validation_predicted_UTS, validation_predicted_EL, validation_predicted_TYS, validation_predicted_RA = validation_predicted.T

# 计算性能指标
def compute_metrics(true, predicted):
    rmse = mean_squared_error(true, predicted, squared=False)
    r2 = r2_score(true, predicted)
    return rmse, r2

# 定义目标变量列表
target_names = ['UTS', 'EL', 'TYS', 'RA']
metrics = {}

for i, target in enumerate(target_names):
    metrics[target] = {}
    # 训练集
    rmse_tr, r2_tr = compute_metrics(train_labels[:, i], train_predicted[:, i])
    metrics[target]['Train_RMSE'] = rmse_tr
    metrics[target]['Train_R2'] = r2_tr
    # 测试集
    rmse_te, r2_te = compute_metrics(test_labels[:, i], test_predicted[:, i])
    metrics[target]['Test_RMSE'] = rmse_te
    metrics[target]['Test_R2'] = r2_te
    # 验证集
    rmse_val, r2_val = compute_metrics(validation_labels[:, i], validation_predicted[:, i])
    metrics[target]['Validation_RMSE'] = rmse_val
    metrics[target]['Validation_R2'] = r2_val

# 输出性能指标
for target in target_names:
    print(f"=== {target} Performance ===")
    print(f"RMSE (Train): {metrics[target]['Train_RMSE']:.2f}")
    print(f"RMSE (Test): {metrics[target]['Test_RMSE']:.2f}")
    print(f"RMSE (Validation): {metrics[target]['Validation_RMSE']:.2f}")
    print(f"R^2 (Train): {metrics[target]['Train_R2']:.2f}")
    print(f"R^2 (Test): {metrics[target]['Test_R2']:.2f}")
    print(f"R^2 (Validation): {metrics[target]['Validation_R2']:.2f}\n")

# 设置绘图参数
plt.rcParams['figure.figsize'] = (20, 15)
plt.rcParams['savefig.dpi'] = 600
plt.rcParams['figure.dpi'] = 600

# 创建子图
fig, axes = plt.subplots(2, 2, figsize=(20, 15))
axes = axes.flatten()

for i, target in enumerate(target_names):
    ax = axes[i]
    ax.set_facecolor("white")
    ax.grid(True, linestyle=':', linewidth=0.9, alpha=0.8)
    for spine in ax.spines.values():
        spine.set_linewidth(1.5)
        spine.set_color('black')
    
    # 确定坐标轴范围
    max_val = max(
        np.max(labels[:, i]),
        np.max([train_predicted[:, i], test_predicted[:, i], validation_predicted[:, i]])) * 1.05
    ax.set_xticks(np.linspace(0, max_val, 11))
    ax.set_yticks(np.linspace(0, max_val, 11))
    
    # 绘制散点
    ax.scatter(train_labels[:, i], train_predicted[:, i], alpha=0.7, s=33, linewidths=0.4, c='#85a2d2',
               edgecolor='#000000', label=f'Training set (R²={metrics[target]["Train_R2"]:.2f})', marker='o')
    ax.scatter(test_labels[:, i], test_predicted[:, i], alpha=0.7, s=33, linewidths=0.4, c='#c2acda',
               edgecolor='#000000', label=f'Testing set (R²={metrics[target]["Test_R2"]:.2f})', marker='o')
    ax.scatter(validation_labels[:, i], validation_predicted[:, i], alpha=0.7, s=33, linewidths=0.4, c='#c0db82',
               edgecolor='#000000', label=f'Validation set (R²={metrics[target]["Validation_R2"]:.2f})', marker='o')
    
    # 绘制参考线
    ax.plot([0, max_val], [0, max_val], 'b--', linewidth=0.8, alpha=0.8)
    ax.set_xlim(0, max_val)
    ax.set_ylim(0, max_val)
    
    # 设置标签和标题
    ax.set_xlabel(f'Real {target}', fontsize=16, weight='bold', labelpad=20, fontname='Times New Roman')
    ax.set_ylabel(f'Predicted {target}', fontsize=16, weight='bold', labelpad=20, fontname='Times New Roman')
    ax.set_title(f'{target}: Real vs Predicted', fontsize=18, weight='bold', fontname='Times New Roman')
    
    # 设置图例
    ax.legend(prop={'family': 'Times New Roman', 'weight': 'bold', 'size': 12}, loc='upper left')

# 添加整体标题
plt.suptitle('Multi-Objective SVR: Real vs Predicted for UTS, EL, TYS, and RA', fontsize=22, weight='bold', fontname='Times New Roman')

# 调整布局
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

# 显示图形
plt.show()

=== UTS Performance ===
RMSE (Train): 100.95
RMSE (Test): 139.46
RMSE (Validation): 113.54
R^2 (Train): 0.94
R^2 (Test): 0.87
R^2 (Validation): 0.91

=== EL Performance ===
RMSE (Train): 6.18
RMSE (Test): 11.13
RMSE (Validation): 9.73
R^2 (Train): 0.93
R^2 (Test): 0.74
R^2 (Validation): 0.71

=== TYS Performance ===
RMSE (Train): 110.47
RMSE (Test): 126.79
RMSE (Validation): 122.19
R^2 (Train): 0.88
R^2 (Test): 0.81
R^2 (Validation): 0.84

=== RA Performance ===
RMSE (Train): 2.30
RMSE (Test): 4.78
RMSE (Validation): 8.34
R^2 (Train): 0.87
R^2 (Test): 0.66
R^2 (Validation): -0.31



ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.